In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import date
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from xgboost import XGBClassifier
import os
import json

path = '/content/drive/MyDrive/TFG ICO/Notebooks/Tables/'

## Choose the number of months corresponding to the *best* training table

In [ ]:
num_months = 36

***
### Configuration for the algorithms

In [ ]:
gs_dict = {
    'logreg': {
        'preprocess': 'standard',
        'classifier': LogisticRegression(random_state=0, max_iter=10000),
        'hyperparameters': [
            {'C': [1e-4, 1e-3, 0.01, 0.1, 1, 10, 100, 1e3, 1e4, 1e5]},
        ]
    },
    'knn': {
        'preprocess': 'minmax',
        'classifier': KNeighborsClassifier(),
        'hyperparameters' : [
            {
                'n_neighbors':[30, 45, 60, 75, 90, 115, 130, 160, 190, 220, 250, 300, 350, 400, 500],
                'weights': ['uniform', 'distance'],
                'p': [1, 2, 3]
            },
        ]
    },
    'svm': {
        'preprocess': 'minmax',
        'classifier': SVC(random_state=0),
        'hyperparameters': [
            {
                'C': [1e-4, 1e-3, 0.01, 0.1, 1, 10, 100, 1e3, 1e4],
                'kernel': ['rbf', 'sigmoid'],
                'gamma': ['auto', 'scale', 1, 2, 4, 8, 16, 32, 64, 128, 256],
            }
        ]
    },
    'rf': {
        'preprocess': None,
        'classifier': RandomForestClassifier(random_state=0),
        'hyperparameters': [{
            'criterion': ['gini', 'entropy'],
            'max_features':[5, 10, 15, 20, 30, 45, 60, None, 'sqrt', 'log2'],
            'max_depth': [None, 2, 3, 4, 5, 10, 20],
            'min_samples_split': [2, 3, 4, 5, 10, 20, 40],
            'min_samples_leaf': [1, 2, 3, 5, 10, 20, 40],
            'n_estimators':[25, 50, 100, 250, 500, 650, 800]
        }]
     },
     'dtree' : {
         'preprocess': None,
         'classifier': DecisionTreeClassifier(random_state=0),
         'hyperparameters': [{
             'criterion': ['gini', 'entropy'],
             'max_depth': [None, 2, 3, 5, 10, 20, 40, 60, 80, 100, 120, 150, 200, 250, 300, 400], 
             'min_samples_split': [2, 3, 5, 8, 10, 20, 30, 40, 60],
             'min_samples_leaf': [1, 2, 3, 5, 7, 10, 15, 20, 40]
          }]
     },
    'xgb': {
        'preprocess': None,
        'classifier': XGBClassifier(random_state=0),
        'hyperparameters': [{
            'n_estimators':[50, 100, 250, 500, 750], 
            'max_depth':[2, 3, 5, 10, 15, 20, 25, 30, 40, 60],
        }]
    }
}

***
# Useful functions

In [ ]:
def load(months):
    df_test = pd.read_csv(path + f'df_test_{months}.csv')
    X_test = df_test.loc[:, df_test.columns != 'is_dead']
    y_test = df_test['is_dead']

    df_train = pd.read_csv(path + f'df_train_{months}.csv')
    X_train = df_train.loc[:, df_train.columns != 'is_dead']
    y_train = df_train['is_dead']
    print(months, 'months')
    print('=== Training set: \n{} patients -- {}% dead \n'.format(len(df_train), round(100*df_train.is_dead.mean(), 1)))

    return X_train, y_train, X_test, y_test


def scale_and_fillna(X_train, X_test):
    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_train_std = pd.DataFrame(X_train_std, columns=X_train.columns)#.fillna(0)
    X_test_std = scaler.transform(X_test)
    X_test_std = pd.DataFrame(X_test_std, columns=X_test.columns)#.fillna(0)

    scaler2 = MinMaxScaler()
    X_train_minmax = scaler2.fit_transform(X_train)
    X_train_minmax = pd.DataFrame(X_train_minmax, columns=X_train.columns)#.fillna(0)
    X_test_minmax = scaler2.transform(X_test)
    X_test_minmax = pd.DataFrame(X_test_minmax, columns=X_test.columns).fillna(0)

    X_train = X_train.fillna(0)
    X_test = X_test.fillna(0)

    return X_train, X_test, X_train_std, X_test_std, X_train_minmax, X_test_minmax


def upsample(X_train, y_train):
    # Merge x and y into one df
    df_train = X_train.copy()
    df_train['is_dead'] = y_train.copy()

    # Separate minority and majority
    df_dead = df_train[df_train.is_dead == 1.]
    df_alive = df_train[df_train.is_dead == 0.]

    # Upsampling the minority
    df_dead_upsampled = resample(df_dead, replace=True, n_samples=len(df_alive), random_state=0)

    # New training set
    df_upsampled = pd.concat([df_alive, df_dead_upsampled])

    X_train_ups = df_upsampled.loc[:, df_upsampled.columns != 'is_dead']
    y_train_ups = df_upsampled['is_dead']
    print('=== After upsample: \n{} patients -- {}% dead'.format(len(y_train_ups), round(100*y_train_ups.mean(), 1)))

    return X_train_ups, y_train_ups


def main(months):
    X_train, y_train, X_test, y_test = load(months)
    X_train, X_test, X_train_std, X_test_std, X_train_minmax, X_test_minmax = scale_and_fillna(X_train, X_test)

    X_train_std, y_train_std = upsample(X_train_std, y_train)
    X_train_minmax, y_train_minmax = upsample(X_train_minmax, y_train)
    X_train, y_train = upsample(X_train, y_train)

    return X_train, X_test, X_train_std, X_test_std, X_train_minmax, X_test_minmax, y_train, y_train_std, y_train_minmax, y_test


X_train, X_test, X_train_std, X_test_std, X_train_minmax, X_test_minmax, y_train, y_train_std, y_train_minmax, y_test = main(num_months)


def fit(algorithm, X, y, cv=5):
    start = time.time()
    
    ### Preprocess
    preprocess = gs_dict[algorithm]['preprocess']
    X = X_train.copy()
    y = y_train.copy()
    if preprocess == 'standard': 
        X = X_train_std.copy()
        y = y_train_std.copy()
    if preprocess == 'minmax': 
        X = X_train_minmax.copy()
        y = y_train_minmax.copy()

    ### Grid search with cross-validation    
    gs = GridSearchCV(
        gs_dict[algorithm]['classifier'], 
        gs_dict[algorithm]['hyperparameters'],
        cv=cv, 
        scoring=['roc_auc','f1','accuracy', 'precision', 'recall'], 
        refit = False
    )
    gs.fit(X, y)    
    df_cv_results = pd.DataFrame(gs.cv_results_)
    df_cv_results['model'] = algorithm
    
    end = time.time()
    duration = end - start
    if duration > 3600:
      duration = duration/3600
      unit = 'hours'
    elif duration > 60:
      duration = duration/60
      unit = 'minutes'
    else:
      unit = 'seconds'
    print('Fitted in {} {}'.format(round(duration, 1), unit))

    cols_to_show = ['model', 'params', 'mean_test_roc_auc', 'mean_test_f1', 'mean_test_precision', 'mean_test_recall']
    df_results = df_cv_results[cols_to_show].round(3)

    return df_results.sort_values(by=['mean_test_roc_auc', 'mean_test_f1'], ascending=False)


# Get grid search results for all algorithms from csv files
def get_global_results():
    df = pd.DataFrame()
    for filename in os.listdir(path):
        if "fit_results_scale1st_36" in filename and ".csv" in filename:
            newdf = pd.read_csv(path + filename)
            df = df.append(newdf, ignore_index=True)

    # Change string dictionary to dictionary by loading as json
    df['params'] = df.params.apply(lambda x: json.loads(x.replace('\'', '\"').replace('None', 'null')))

    return df.sort_values(by=['mean_test_roc_auc', 'mean_test_f1'], ascending=False)


def plot_roc_curve(y_test, y_pred_prob):
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    plt.plot(fpr, tpr, linewidth=2, label=None)
    plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
    plt.grid(True)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate (Recall)')
    plt.show()


def evaluate_best_models(df_results, X_train, X_test):
  df_gs_results = pd.DataFrame()
  df_final_results = pd.DataFrame(columns=['model', 'roc_auc', 'f1', 'accuracy', 'precision', 'recall'])

  for i, alg in enumerate(df_results.model.unique()):
    print('=====', alg, '=====')

    # Detect best results for the algorithm
    best_result_alg = df[df.model == alg].reset_index().iloc[0]
    df_gs_results = df_gs_results.append(best_result_alg, ignore_index=True)
    best_hyperparameters = best_result_alg['params']
    print('Best hyperparameters:', best_hyperparameters)

    if alg == 'svm':
      best_hyperparameters.update({'probability': True})

    preprocess = gs_dict[alg]['preprocess']
    if preprocess == 'standard': 
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        #gs_dict[algorithm].update({'scaler_used': scaler})
    if preprocess == 'minmax': 
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        #gs_dict[algorithm].update({'scaler_used': scaler})

    try:
        X_train = X_train.fillna(0)
    except:
        X_train = np.nan_to_num(X_train, nan=0)

    # Select and train the best estimator
    model = gs_dict[alg]['classifier'].set_params(**best_hyperparameters)
    model.fit(X_train, y_train)
    print('third')

    #Make predictions on the test set
    #scaler = gs_dict[alg].get('scaler_used')
    if preprocess is not None:
      X_test = scaler.transform(X_test)
    try:
      X_test = X_test.fillna(0)
    except:
      X_test = np.nan_to_num(X_test, nan=0)
    y_pred = model.predict(X_test)
    print('Predicted', sum(y_pred), 'deaths out of', len(y_pred), 'patients ({}%)'.format(round(100*sum(y_pred)/len(y_pred),1)))

    y_pred_prob = model.predict_proba(X_test)[:, 1]
    print('Min prob:', round(min(y_pred_prob), 3))
    print('Max prob:', round(max(y_pred_prob), 3))

    plot_roc_curve(y_test, y_pred_prob)

    ## Metrics
    df_final_results.loc[i, 'model'] = alg
    df_final_results.loc[i, 'roc_auc'] = round(roc_auc_score(y_test, y_pred_prob), 3)
    df_final_results.loc[i, 'f1'] = round(f1_score(y_test, y_pred), 3)
    df_final_results.loc[i, 'accuracy'] = round(accuracy_score(y_test, y_pred), 3)
    df_final_results.loc[i, 'precision'] = round(precision_score(y_test, y_pred), 3)
    df_final_results.loc[i, 'recall'] = round(recall_score(y_test, y_pred), 3)

  df_final_results.to_csv(path + f'final_results_scale1st_{num_months}.csv', index=False)
  return df_gs_results, df_final_results

### Fit a single model

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
algorithm = 'xgb'

fit_results = fit(algorithm, X_train, y_train)
fit_results.to_csv(path + f'fit_results_scale1st_{num_months}_{algorithm}.csv', index=False)
fit_results

# Choose the best model for each algorithm and evaluate with test set

In [ ]:
# Use this if loading fit results from different files
df = get_global_results()

gs_results, test_results = evaluate_best_models(df, X_train, X_test)

### Metrics on the test set

In [ ]:
test_results.sort_values(by=['roc_auc', 'f1'], ascending=False)

### Metrics from cross-validation

In [ ]:
names_dict = {'mean_test_roc_auc': 'roc_auc', 'mean_test_f1': 'f1', 'mean_test_precision': 'precision', 'mean_test_recall': 'recall'}
gs_results = gs_results.rename(columns=names_dict)
cols_order = ['model', 'roc_auc', 'f1', 'precision', 'recall', 'params']
gs_results[cols_order].sort_values(by=['roc_auc', 'f1'], ascending=False)